In [57]:
import numpy as np
import pandas as pd
import os
import shutil
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import optimizers
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

In [5]:
labels = pd.read_csv("data/train.csv")

In [6]:
labels

,image_id,category
0,0,77
1,1,81
2,2,52
3,3,72
4,4,58
5,5,2
6,6,75
7,7,67
8,8,51
9,9,15


In [21]:
origtrain = "train/"
newtrain = "Train/"

In [36]:
#creating trainset
for imagefile,classname in labels.values:
  #  print(imagefile,classname)
    if not os.path.exists(newtrain + str(classname)):
        os.mkdir(newtrain+str(classname))
    srcfile = origtrain + str(imagefile) + ".jpg"
    destfile = newtrain  +str(classname) + "/" + str(imagefile)+".jpg"
    
    try:
        shutil.copy(srcfile, destfile)
    except IOError as e:
        print('Unable to copy file {} to {}'
              .format(srcfile, destfile))
    except:
        print('When try copy file {} to {}, unexpected error: {}'
              .format(srcfile, destfile, sys.exc_info()))
        

In [30]:
valset ="Val/"
num_imgs =10

In [39]:
#creating valset
for i in range(1,103):
    if not os.path.exists(valset + str(i)):
        os.mkdir(valset+str(i))
    images = os.listdir(newtrain + str(i))
    #print(images)
    for j,image in enumerate(images):
        if(j<num_imgs):
            print(i)
            srcfile = newtrain + str(i) +"/" + image
            destfile= valset + str(i) + "/"  
            try:
                shutil.move(srcfile, destfile)
                #print("image moved {}".format(image))
            except IOError as e:
                print('Unable to move file {} to {}'
                  .format(srcfile, destfile))
            except:
                print('When try move file {} to {}, unexpected error: {}'
                  .format(srcfile, destfile, sys.exc_info()))


1
image moved 10403.jpg
1
image moved 17390.jpg
1
image moved 16961.jpg
1
image moved 1336.jpg
1
image moved 15759.jpg
1
image moved 12740.jpg
1
image moved 979.jpg
1
image moved 11742.jpg
1
image moved 5512.jpg
1
image moved 4908.jpg
2
image moved 15902.jpg
2
image moved 12050.jpg
2
image moved 17000.jpg
2
image moved 14747.jpg
2
image moved 6149.jpg
2
image moved 6177.jpg
2
image moved 8648.jpg
2
image moved 4619.jpg
2
image moved 12615.jpg
2
image moved 14760.jpg
3
image moved 3925.jpg
3
image moved 16102.jpg
3
image moved 14566.jpg
3
image moved 4161.jpg
3
image moved 6106.jpg
3
image moved 13837.jpg
3
image moved 4669.jpg
3
image moved 4548.jpg
3
image moved 3781.jpg
3
image moved 1025.jpg
4
image moved 13453.jpg
4
image moved 6167.jpg
4
image moved 4982.jpg
4
image moved 395.jpg
4
image moved 16761.jpg
4
image moved 12026.jpg
4
image moved 8422.jpg
4
image moved 5767.jpg
4
image moved 11123.jpg
4
image moved 11017.jpg
5
image moved 4128.jpg
5
image moved 12357.jpg
5
image moved 1

In [54]:
imagesize = 250;
num_training = 17516
num_val = 1020
batch_size = 50
no_epochs = 25

In [44]:
model = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(250, 250, 3))

In [45]:
for layer in model.layers:
    layer.trainable = False

In [46]:
x = model.output
x = Flatten()(x)
x = Dense(128,activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(128, activation="relu")(x)
predictions = Dense(102, activation="softmax")(x)

W0725 15:27:41.198462 139642407294720 deprecation.py:506] From /home/justdial/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [47]:
model_final = Model(input = model.input, output = predictions)

/home/justdial/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [51]:
model_final.compile(loss = "categorical_crossentropy", 
                    optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
                    metrics=["accuracy"])

W0725 15:31:07.681152 139642407294720 deprecation_wrapper.py:119] From /home/justdial/miniconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [55]:
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.3,
width_shift_range = 0.3,
height_shift_range=0.3,
rotation_range=30)

train_generator = train_datagen.flow_from_directory(
newtrain,
target_size = (250, 250),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
valset,target_size = (250, 250),class_mode = "categorical")



Found 17516 images belonging to 102 classes.
Found 1020 images belonging to 102 classes.


In [58]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("InceptionResNetV2_clean_dataset_R_S_W_B.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [60]:
model_final.fit_generator(
train_generator,
samples_per_epoch = num_training,
epochs = no_epochs,
validation_data = validation_generator, 
nb_val_samples = num_val,
callbacks = [checkpoint, early])

/home/justdial/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  import sys
/home/justdial/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., epochs=25, validation_data=<keras_pre..., callbacks=[<keras.ca..., steps_per_epoch=350, validation_steps=1020)`
  import sys
W0725 16:32:55.967046 139642407294720 deprecation.py:323] From /home/justdial/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (fr

Epoch 1/25


KeyboardInterrupt: 